# 🎯 TicketCluster — Unsupervised Customer Support Ticket Intelligence

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RansiluRanasinghe/TicketCluster-Unsupervised-ML/blob/main/notebook.ipynb)

---

## 📌 Project Overview

This notebook implements **TicketCluster**, an unsupervised NLP system that analyzes and groups customer support tickets using **K-Means clustering**.

**Goal:** Simulate how real organizations explore unlabeled support tickets to identify recurring issues, emerging patterns, and operational insights—without predefined categories.

---

## 🎯 Workflow

1. **Minimal, production-oriented text preprocessing**
2. **Vectorization** of ticket content (Subject + Body)
3. **Unsupervised clustering** to discover natural groupings
4. **Cluster interpretation** from business perspectives

This notebook emphasizes **model reasoning and reliability** over aggressive tuning, reflecting real customer-support analytics pipelines.

---

**Author:** Ransilu Ranasinghe | [GitHub](https://github.com/RansiluRanasinghe) | [LinkedIn](https://www.linkedin.com/in/ransilu-ranasinghe-a596792ba)

---